
# Course Environment Setup (Conda + ObsPy Stack)

This notebook installs the **core scientific Python stack** used throughout the course:

- NumPy
- Matplotlib
- Pandas
- ObsPy

We will:
1. Generate a **bash install script**
2. Display the script in this notebook
3. Run it from Python

**Why this structure?**
- The notebook documents the workflow (reproducibility)
- Bash does environment management (the right tool for the job)
- The script can be reused later without the notebook


In [ ]:

from pathlib import Path
import textwrap

# ---- EDIT THESE IF NEEDED ----
ENV_NAME = "seismo"
PYTHON_VERSION = "3.11"
PACKAGES = ["numpy", "matplotlib", "pandas", "obspy", "ipykernel"]
# -----------------------------

script_path = Path("scripts/install_seismo_env.sh")
script_path.parent.mkdir(parents=True, exist_ok=True)

print("Environment:", ENV_NAME)
print("Python version:", PYTHON_VERSION)
print("Packages:", ", ".join(PACKAGES))


In [ ]:

bash_script = textwrap.dedent(f"""
#!/usr/bin/env bash
set -euo pipefail

ENV_NAME="{ENV_NAME}"
PYTHON_VERSION="{PYTHON_VERSION}"
PACKAGES="{ ' '.join(PACKAGES) }"

echo "=== Conda environment setup ==="
echo "Env: $ENV_NAME"
echo "Python: $PYTHON_VERSION"
echo "Packages: $PACKAGES"
echo

if ! command -v conda >/dev/null 2>&1; then
  echo "ERROR: conda not found. Restart terminal or source Miniconda."
  exit 1
fi

conda config --add channels conda-forge >/dev/null 2>&1 || true
conda config --set channel_priority strict

if conda env list | awk '{print $1}' | grep -qx "$ENV_NAME"; then
  echo "Env already exists."
else
  conda create -y -n "$ENV_NAME" python="$PYTHON_VERSION"
fi

conda install -y -n "$ENV_NAME" $PACKAGES

conda run -n "$ENV_NAME" python -m ipykernel install --user --name "$ENV_NAME" --display-name "Python ($ENV_NAME)"

echo "Done."
""")

print(bash_script)


In [ ]:

import os

script_path.write_text(bash_script, encoding="utf-8")
os.chmod(script_path, 0o755)

print("Wrote installer script to:", script_path)



## Run the installer

This will create the conda environment and install packages.


In [ ]:

import subprocess

cmd = ["bash", str(script_path)]
print("Running:", " ".join(cmd))

subprocess.run(cmd, check=False)


In [ ]:

verify_cmd = [
    "conda", "run", "-n", ENV_NAME, "python", "-c",
    "import numpy, pandas, matplotlib, obspy; print('All imports OK')"
]

subprocess.run(verify_cmd, check=True)



## Using this environment in VS Code

1. Restart VS Code
2. Open a notebook
3. Select kernel → **Python (seismo)**

If the kernel does not appear, rerun the installer.
